# Creating the MCS (Hansen, Lunde, Nason ; 2011)

In [ ]:
#######################################################################
#######################################################################
######################### Importing Packages ##########################
#######################################################################
#######################################################################

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import tensorflow as tf
import math
from datetime import datetime, timedelta
import scipy.stats

# need to clean these up later
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow_probability as tfp
import pandas as pd
from keras.models import Sequential
from tensorflow.keras import regularizers
from keras.layers import Activation, Input, Dense, LSTM, concatenate
from keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
tfd = tfp.distributions

from numpy.random import rand
from numpy import ix_
np.random.seed(1337)

# Importing Data

In [ ]:
# import Neural network and GARCH models.

# The MCS 

In [ ]:

def gw(L: np.array,
       tau: int,
       H: Optional[np.array] = None,
       kernel: Optional[Union[str, Callable]] = None,
       bw: Optional[int] = None,
       kernel_kwargs: Optional[dict] = None,
       alpha: float = 0.05) -> tuple[float, float, float]:
    """
    Test of Equal Conditional Predictive Ability by Giacomini and White (2006).
    Used here for testing and debugging but made available through the package interface.
    This is a reimplementation from the MATLAB code provided by
    Giacomini (https://gist.github.com/ogrnz/91f37140011d1c2447934766274c4070)
    References:
        - Giacomini, R., & White, H. (2006). Tests of conditional predictive ability. Econometrica, 74(6), 1545-1578.
    :param L: (Tx2) array of forecast losses
    :param H: (Txq) array of instruments. If `None` provided, defaults to the unconditional EPA (DM test)
    :param tau: Forecast horizon
    :param kernel: (default `None`)
        If multistep forecast (`tau` > 1), covariance matrix is an HAC estimator.
        Original implementation uses a Bartlett kernel with bandwidth `tau - 1`.
        If a `str`, must match one of `arch` package variance estimator:
         > https://arch.readthedocs.io/en/latest/covariance/covariance.html
        If a `Callable`, must simply return a (qxq) covariance matrix (see arg `H`).
    :param bw: (default `None`)
        Bandwidth of the `kernel`. Typically set to `forecasting horizon - 1`.
        If set to `None`, will let the kernel compute the optimal bandwidth if supported.
    :param kernel_kwargs: (default `None`)
        An optional dict of `argument: value` passed to `kernel`.
        If `kernel` is a `Callable`, the eventual bandwidth must be passed here.
    :param alpha: Significance level
    :return: tuple(S, crit_val, p_val)
        S: test statistic,
        cval: critical value for significance lvl,
        pval: p-value of test
    """
    T = L.shape[0]  # Number of observations
    d = L[:, 0] - L[:, 1]  # Loss differential
    if H is None:  # Instruments (defaults to unconditional EPA)
        H = np.ones((T, 1))
    q = H.shape[1]

    reg = np.empty(H.shape)
    for jj in range(H.shape[1]):
        reg[:, jj] = H[:, jj] * d

    if tau == 1:  # One-step
        beta = np.linalg.lstsq(reg, np.ones(T), rcond=None)[0][0].item()
        res = np.ones(T) - beta * reg
        r2 = 1 - np.mean(res ** 2)
        S = T * r2
    else:  # Multistep
        omega = np.empty((q, q))  # Defined here to make linter happy and inform about dims
        if isinstance(kernel, Callable):  # Custom callable
            omega = kernel(reg, **kernel_kwargs)
        elif isinstance(kernel, str):  # Arch covariance
            kerfunc = getattr(kernels, kernel)
            ker = kerfunc(reg, bandwidth=bw, **kernel_kwargs)
            omega = ker.cov.long_run
        else:
            raise NotImplementedError
        zbar = reg.mean().T
        S = (T * zbar.T * np.linalg.pinv(omega) * zbar).item()

    dof = reg.shape[1]
    cval = chi2.ppf(1 - alpha, dof)
    pval = 1 - chi2.cdf(abs(S), dof)

    return S, cval, pval


def mgw(L: np.array,
        H: Optional = None,
        covar_style: Literal["sample", "hac"] = "sample",
        kernel: Optional[Union[str, Callable]] = None,
        bw: Optional[int] = None,
        kernel_kwargs: Optional[dict] = None,
        alpha: float = 0.05):
    """
    Implements the multivariate Giacomini-White (MGW) (Borup et al., 2022) test of equal predictive ability.
    This is a reimplementation from the MATLAB code provided by
    Borup (https://sites.google.com/view/danielborup/research)
    Notes:
        If only 2 models are compared, it reduces to the Giacomini-White test (GW) (Giacomini and White, 2006)
        If further no conditioning information H is given, it reduces to the
        original Diebold-Mariano test (DM) (Diebold and Mariano, 1995)
        If more than 2 models are compared but with no conditioning information H,
        it reduces to multivariate Diebold-Mariano (MDM) (Mariano and Preve, 2012)
    References:
        - Borup, Daniel and Eriksen, Jonas Nygaard and Kjær, Mads Markvart and Thyrsgaard, Martin,
        Predicting Bond Return Predictability. Available at http://dx.doi.org/10.2139/ssrn.3513340
        - Diebold, F.X., and R.S. Mariano (1995) ‘Comparing Predictive Accuracy,’ Journal
        of Business and Economic Statistics 13, 253–263.
        - Giacomini, R., & White, H. (2006). Tests of conditional predictive ability.
        Econometrica, 74(6), 1545-1578.
        - Mariano, R. S., & Preve, D. (2012). Statistical tests for multiple forecast comparison.
        Journal of econometrics, 169(1), 123-130.
    :param L:
        Txk matrix of losses of k models with T forecasts.
    :param H: (default `None`)
        Txq matrix of a constant and information set (test function).
        If not provided, set to a (Tx1) column vector of 1, amounts to the
        unconditional MWG test, which is equivalent to the multivariate Diebold-Mariano (Mariano and Preve, 2012).
    :param covar_style: (default 'sample')
        How to compute the covariance matrix.
        Either the sample covariance ('sample') or an HAC estimator ('hac').
    :param kernel: (default `None`)
        If covariance matrix is an HAC estimator, what type to compute.
        If a `str`, must match one of `arch` package variance estimator:
         > https://arch.readthedocs.io/en/latest/covariance/covariance.html
        If a `Callable`, must simply return a
    :param bw: (default `None`)
        Bandwidth of the `kernel`. Typically set to `forecasting horizon - 1`.
        If set to `None`, will let the kernel compute the optimal bandwidth if supported.
    :param kernel_kwargs: (default `None`)
        An optional dict of `argument: value` passed to `kernel`.
        If `kernel` is a `Callable`, the eventual bandwidth must be passed here.
    :param alpha: (default 0.05)
        Significance level.
    :return: tuple(S, cval, pval)
        S: float, the computed test statistic
        cval: float, the corresponding critical value
        pval: float, the p-value of S.
    """
    # Args checks
    if kernel is not None and covar_style == "sample":
        raise ValueError(f"{kernel=} incompatible with {covar_style=}.")
    if kernel is None and covar_style == "hac":
        raise ValueError("Set `kernel` when using an HAC estimator.")
    if bw is not None and covar_style == "sample":
        raise ValueError(f"{bw=} incompatible with {covar_style=}.")
    if L.shape[1] < 2:
        raise ValueError(f"Not enough columns for matrix of  losses {L.shape[1]=}.")

    if kernel_kwargs is None:
        kernel_kwargs = {}

    # Initialize
    T = L.shape[0]
    p = L.shape[1] - 1
    if H is None:  # defaults to unconditional EPA
        H = np.ones((T, 1))

    # Loss differentials
    D = np.diff(L, axis=1)

    # Conditioning information
    q = H.shape[1]
    reg = np.empty((T, q * p))

    for i in range(T):
        reg[i, :] = np.kron(H[i, :], D[i, :])

    Dbar = np.mean(reg, axis=0)

    # Compute covar matrix
    omega = np.empty((q * p, q * p))  # Defined here to make linter happy and inform about dims
    if covar_style == "sample":
        omega = (reg - Dbar).T @ (reg - Dbar) / (T - 1)
    elif covar_style == "hac":  # HAC estimator
        if isinstance(kernel, Callable):  # Custom callable
            omega = kernel(reg, **kernel_kwargs)
        elif isinstance(kernel, str):  # Arch covariance
            kerfunc = getattr(kernels, kernel)
            ker = kerfunc(reg, bandwidth=bw, **kernel_kwargs)
            omega = ker.cov.long_run
        else:
            raise NotImplementedError
    else:
        raise NotImplementedError

    # Compute statistic
    dof = q * p
    S = (T * Dbar @ np.linalg.pinv(omega) @ Dbar.T).item()
    cval = chi2.ppf(1 - alpha, dof)
    pval = 1 - chi2.cdf(S, dof)

    return S, cval, pval


def cmcs(L: np.array, H: Optional = None, alpha: float = 0.05, **kwargs):
    """
    Perform the Conditional Model Confidence Set (CMCS).
    The MCS procedure from Hansen (2011) is adapted to use MGW (Borup et al., 2022)
    instead of bootstrapping the critical values. Allows to reduce an initial set of models to a
    set of models with equal (conditional) predictive ability.
    Also, allows to use conditioning information (`H`, hence the 'Conditional'),
    to get the best MCS based on expected future loss.
    This is a reimplementation from the MATLAB code provided by
    Borup (https://sites.google.com/view/danielborup/research)
    References:
        - Borup, Daniel and Eriksen, Jonas Nygaard and Kjær, Mads Markvart and Thyrsgaard, Martin,
        Predicting Bond Return Predictability. Available at http://dx.doi.org/10.2139/ssrn.3513340
        - Hansen, P. R., Lunde, A., & Nason, J. M. (2011). The model confidence set. Econometrica, 79(2), 453-497.
    :param L:
        (Txk) matrix of losses of k models with T forecasts.
    :param H: (default `None`)
        (Txq) matrix of a constant and information set (test function).
        If not provided, set to a (Tx1) column vector of 1, amounts to the
        unconditional MWG test, which is equivalent to the multivariate Diebold-Mariano (Mariano and Preve, 2012).
    :param alpha: (default 0.05)
        Significance level used in the MGW test.
    :param **kwargs: Arguments passed to `feval.mgw`. Usually define covariance estimator and such.
    :return: tuple(mcs, S, cval, pval, removed)
        mcs: (1xk) np.array where models included in the best model confidence set are noted as 1.
        S: float, the computed test statistic of the last test.
        cval: float, the corresponding critical value.
        pval: float, the p-value of S.
        removed: (1xk) np.array where a column represents an algorithm cycle.
            That way, we can see which model index was removed at which iteration.
    """
    # Initialize
    T = L.shape[0]
    k = L.shape[1]
    if H is None:
        H = np.ones((T, 1))

    # Init loop
    S, cval, pval = np.inf, 1, 1
    mcs = np.ones((1, k))
    removed = np.zeros((1, k))

    j = 0
    while S > cval:
        # Create L_to_use, the losses of models still in MCS
        L_to_use = L[:, (mcs == 1)[0]]

        if L_to_use.shape[1] == 1:  # Only 1 model left in set
            break

        # Perform MGW
        S, cval, pval = mgw(L_to_use, H, alpha=alpha, **kwargs)

        # H0 still rejected, apply elimination criterion
        if S > cval:
            mcs, removed[0, j] = elim_rule(L, mcs, H)

        j += 1

    return mcs, S, cval, pval, removed


def elim_rule(L: np.array,
              mcs: np.array,
              H: Optional = None):
    """
    Elimination rule that allows to rank losses based on expected future loss given the information set `H`.
    If `H` is a vector of constant, it amounts to ranking losses based on average loss.
    See Borup et al. (2022) and Hansen (2011).
    This is a reimplementation from the MATLAB code provided by
    Borup (https://sites.google.com/view/danielborup/research)
    References:
        - Borup, Daniel and Eriksen, Jonas Nygaard and Kjær, Mads Markvart and Thyrsgaard, Martin,
        Predicting Bond Return Predictability. Available at http://dx.doi.org/10.2139/ssrn.3513340
        - Hansen, P. R., Lunde, A., & Nason, J. M. (2011). The model confidence set. Econometrica, 79(2), 453-497.
    :param L:
        (Txk) matrix of losses of k models with T forecasts.
    :param mcs:
        (1xk) vector of current model confidence set, where the least performing model will be eliminated.
    :param H: (default `None`)
        (Txq) matrix of a constant and information set (test function).
    :return: tuple(mcs, removed)
        mcs: (1xk) np.array where models included in the best model confidence set are noted as 1.
        removed: (1xk) np.array where a column represents an algorithm cycle.
            That way, we can see which model index was removed at which iteration.
    """
    # Initialize
    k = mcs.shape[1]
    q = H.shape[1]
    new_k = np.count_nonzero(mcs)

    if L.shape[1] != k:
        raise ValueError(f"Dimensions of {L.shape[1]=} do not match {mcs.shape[1]=}.")

    L_to_use = np.zeros((L.shape[0], new_k))
    curr_set = np.zeros((1, new_k))
    j = 0
    for i in range(k):  # TODO could be simplified?
        if mcs[0, i] == 1:
            L_to_use[:, j] = L[:, i]
            curr_set[0, j] = i
            j += 1

    combinations = np.arange(0, j).reshape(1, -1)  # TODO why matrix? could be vect
    L_hat = np.zeros(combinations.shape)

    # Estimate
    for j in range(combinations.shape[0]):  # TODO no loop if vect
        L_intra_use = L_to_use[:, combinations[j, :]]  # TODO directly call j?

        deltas = np.zeros((q, new_k - 1))
        for i in range(L_to_use.shape[1] - 1):
            Y_used = L_intra_use[:, i + 1] - L_intra_use[:, i]
            Y_used = Y_used.reshape(-1, 1)
            deltas[:, i] = (np.linalg.inv(H.T @ H) @ H.T @ Y_used).reshape(-1, )

        delta_L_hat = (deltas.T @ H[-1, :].T).reshape(-1, 1)
        starting_point = combinations[combinations == 0]  # should always return 1 idx

        # Normalize
        vL_hat = np.zeros(L_hat.shape)
        vL_hat[0, 0] = 1
        for i in range(L_to_use.shape[1] - 1):
            vL_hat[0, i + 1] = vL_hat[0, i] + delta_L_hat[i, 0]
        L_hat[j, :] = np.divide(vL_hat, vL_hat[0, starting_point].item())

    # Rank losses
    indx = np.argmax(L_hat)
    col = np.unique(combinations[0, indx])

    # Update mcs
    mcs[0, curr_set[0, col].astype(int)] = 0
    removed = curr_set[0, col]
    return mcs, removed

In [ ]:
# calculate the MCS here

# Inference

In [ ]:
# explain which models are in the MCS here